In [1]:
%%bash 

today=$(date '+%Y-%m-%d')

# wget -q https://www.ebi.ac.uk/gwas/api/search/downloads/alternative \
#     -O gwas_associations_${today}.tsv
    
# wget -q https://www.ebi.ac.uk/gwas/api/search/downloads/studies_alternative \
#     -O gwas_studies_${today}.tsv
    
wget -q  https://www.ebi.ac.uk/gwas/api/search/downloads/ancestry    \
    -O gwas_ancestries_${today}.tsv

ls -lah *tsv

-rw-r--r--  1 dsuveges  384566875    10M 23 May 22:06 gwas_ancestries_2022-05-23.tsv
-rw-r--r--  1 dsuveges  384566875   227M 20 May 06:31 gwas_associations_2022-05-20.tsv
-rw-r--r--  1 dsuveges  384566875   227M 23 May 16:41 gwas_associations_2022-05-23.tsv
-rw-r--r--  1 dsuveges  384566875    17M 20 May 06:31 gwas_studies_2022-05-20.tsv
-rw-r--r--  1 dsuveges  384566875    17M 23 May 16:41 gwas_studies_2022-05-23.tsv


In [3]:
import pyspark.sql
import pyspark.sql.types as t
import pyspark.sql.functions as f


spark = (pyspark.sql.SparkSession
       .builder
       .master("local[*]")
       .getOrCreate()
)
        
# Reading GWAS Catalog data into spark dataframe
gwas_df = spark.read.csv('gwas_associations_2022-05-20.tsv', sep='\t', header=True).persist()

print(gwas_df.count())
gwas_df.show(1,False,True)
gwas_df.printSchema()


Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.AssertionError: assertion failed: Expected hostname or IPv6 IP enclosed in [] but got fe80:0:0:0:ff98:a2e4:f967:1ff7%14
	at scala.Predef$.assert(Predef.scala:223)
	at org.apache.spark.util.Utils$.checkHost(Utils.scala:1072)
	at org.apache.spark.executor.Executor.<init>(Executor.scala:89)
	at org.apache.spark.scheduler.local.LocalEndpoint.<init>(LocalSchedulerBackend.scala:64)
	at org.apache.spark.scheduler.local.LocalSchedulerBackend.start(LocalSchedulerBackend.scala:132)
	at org.apache.spark.scheduler.TaskSchedulerImpl.start(TaskSchedulerImpl.scala:220)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:579)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstruct

In [32]:
print('Study count: {}'.format(gwas_df.select('STUDY ACCESSION').distinct().count()))
print('Split count by variants:')
print('   By CHR_ID: {}'.format(gwas_df.filter(f.col('CHR_ID').contains(';')).count()))
print('   By CHR_POS: {}'.format(gwas_df.filter(f.col('CHR_POS').contains(';')).count()))
print('   By CHR_ID or CHR_POS: {}'.format(gwas_df.filter(f.col('CHR_ID').contains(';') | f.col('CHR_POS').contains(';')).count()))
print('   By SNPS: {}'.format(gwas_df.filter(f.col('SNPS').contains(';')).count()))
print('   By STRONGEST SNP-RISK ALLELE: {}'.format(gwas_df.filter(f.col('STRONGEST SNP-RISK ALLELE').contains(';')).count()))



Study count: 15844
Split count by variants:
   By CHR_ID: 220
   By CHR_POS: 220
   By CHR_ID or CHR_POS: 220
   By SNPS: 1956
   By STRONGEST SNP-RISK ALLELE: 1956


In [47]:
(
    gwas_df
    .filter(f.col('CHR_POS').contains(';'))
    .select(
        f.split(f.col('CHR_ID'), ';').alias('CHR'), 
        f.split(f.col('CHR_POS'), ';').alias('POS'), 
        f.split(f.col("SNPS"), ';').alias('SNPS'),
        f.col('SNP_ID_CURRENT'),
        f.col('STUDY ACCESSION'),
        f.col('STRONGEST SNP-RISK ALLELE')
    )
    .filter(f.size('POS') != f.size('SNPS'))
    .show(1, vertical=True, truncate=False)
)

-RECORD 0-----------------------------------------------------------------------
 CHR                       | [6, 6]                                             
 POS                       | [32353777, 32421871]                               
 SNPS                      | [rs2395148,  rs2856683,  rs3135363,  rs9357152]    
 SNP_ID_CURRENT            | null                                               
 STUDY ACCESSION           | GCST000408                                         
 STRONGEST SNP-RISK ALLELE | rs2395148-A; rs2856683-C; rs3135363-A; rs9357152-A 
only showing top 1 row



In [48]:
(
    gwas_df
    .filter(
        f.col('SNPS').contains(';')
        & f.col('CHR_POS').isNull()
    )
    .select('CHR_ID', 'CHR_POS', 'SNPS', 'STUDY ACCESSION', 'SNP_ID_CURRENT', f.col('STRONGEST SNP-RISK ALLELE'))
    .show(3, False, True)
)

-RECORD 0-------------------------------------------------------------------
 CHR_ID                    | null                                           
 CHR_POS                   | null                                           
 SNPS                      | rs17435444; rs13116936                         
 STUDY ACCESSION           | GCST001879                                     
 SNP_ID_CURRENT            | null                                           
 STRONGEST SNP-RISK ALLELE | rs17435444-A; rs13116936-G                     
-RECORD 1-------------------------------------------------------------------
 CHR_ID                    | null                                           
 CHR_POS                   | null                                           
 SNPS                      | SNP_A-2032347; SNP_A-2196879; rs10871290       
 STUDY ACCESSION           | GCST000187                                     
 SNP_ID_CURRENT            | null                                           

In [16]:
(
    gwas_df
    .filter(
        f.col('CHR_POS').contains(';')
    )
    .show(3, False, True)
)

-RECORD 0-----------------------------------------------------------------------------------------------------------------------------------------------------
 DATE ADDED TO CATALOG      | 2011-01-14                                                                                                                      
 PUBMEDID                   | 20801717                                                                                                                        
 FIRST AUTHOR               | Shatunov A                                                                                                                      
 DATE                       | 2010-10-01                                                                                                                      
 JOURNAL                    | Lancet Neurol                                                                                                                   
 LINK                       | www.ncbi.nlm.nih

In [22]:
#     # Split rows by multiple variants
#     gwas['CHR_ID'] = gwas['CHR_ID'].astype(str).str.split(';')
#     gwas['CHR_POS'] = gwas['CHR_POS'].astype(str).str.split(';')
#     gwas['SNPS'] = gwas['SNPS'].astype(str).str.split('; ')
#     gwas['STRONGEST SNP-RISK ALLELE'] = gwas['STRONGEST SNP-RISK ALLELE'].astype(str).str.split('; ')
gwas_split = (
    gwas_df
    .withColumn('CHR_ID', f.split('CHR_ID', ';'))
    .withColumn('CHR_POS', f.split('CHR_POS', ';'))
    .withColumn('SNPS', f.split('SNPS', '; '))
    .withColumn('STRONGEST SNP-RISK ALLELE', f.split('STRONGEST SNP-RISK ALLELE', '; '))
)

(
    gwas_split
    .filter(f.size('CHR_POS') != f.size('STRONGEST SNP-RISK ALLELE'))
    .show(10, False, True)
)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------
 DATE ADDED TO CATALOG      | 2020-05-19                                                                                                                        
 PUBMEDID                   | 31926482                                                                                                                          
 FIRST AUTHOR               | Shen H                                                                                                                            
 DATE                       | 2020-01-11                                                                                                                        
 JOURNAL                    | Neuropsychopharmacology                                                                                                           
 LINK                       | www.

In [23]:
import pandas as pd

# Load gwas catalog data
gwas_raw = pd.read_csv(
    'gwas_associations_2022-05-20.tsv',
    sep='\t',
    header=0,
    dtype={'P-VALUE': str},
    low_memory=False,
    # na_values=['']
)

# Set a row (association) id -> Seriously... the table is alrady indexed.
gwas_raw['assoc_id'] = list(range(1, gwas_raw.shape[0] + 1))
gwas = gwas_raw.copy()

# Split rows by multiple variants
gwas['CHR_ID'] = gwas['CHR_ID'].astype(str).str.split(';')
gwas['CHR_POS'] = gwas['CHR_POS'].astype(str).str.split(';')
gwas['SNPS'] = gwas['SNPS'].astype(str).str.split('; ')
gwas['STRONGEST SNP-RISK ALLELE'] = gwas['STRONGEST SNP-RISK ALLELE'].astype(str).str.split('; ')

print(len(gwas))

372752


In [54]:
(
    gwas_df
    .filter(f.lower(f.col('SNPS')).contains('chr'))
    .count()
)

10387

In [55]:
10387 / gwas_df.count() * 100

2.7865712323475127

### Ancestries

In [4]:
ancestries = spark.read.csv('gwas_ancestries_2022-05-23.tsv', sep='\t', header=True).persist()
print(ancestries.show(1, False, True))
print(ancestries.count())
print(ancestries.select('STUDY ACCESSION').distinct().count())



-RECORD 0---------------------------------------------------------------------
 STUDY ACCESSION                | GCST009498                                  
 PUBMEDID                       | 31681408                                    
 FIRST AUTHOR                   | Yang XL                                     
 DATE                           | 2019-10-11                                  
 INITIAL SAMPLE DESCRIPTION     | up to 182,902 European ancestry individuals 
 REPLICATION SAMPLE DESCRIPTION | NA                                          
 STAGE                          | initial                                     
 NUMBER OF INDIVDUALS           | 182902                                      
 BROAD ANCESTRAL CATEGORY       | European                                    
 COUNTRY OF ORIGIN              | NR                                          
 COUNTRY OF RECRUITMENT         | NR                                          
 ADDITONAL ANCESTRY DESCRIPTION | null              

In [7]:
import pandas as pd
import requests 


df = 
df.head()

,assoc_files,category,category_index,gc_lambda,num_cases,num_cases_prev,num_controls,num_gw_significant,num_gw_significant_prev,phenocode,phenostring
0,[/mnt/nfs/pheweb/r6/generated-by-pheweb/pheno_...,I Certain infectious and parasitic diseases (A...,1,"{'0.001': 0.98843, '0.01': 0.98437, '0.1': 0.9...",102,87,234437,0,0,AB1_AMOEBIASIS,Amoebiasis
1,[/mnt/nfs/pheweb/r6/generated-by-pheweb/pheno_...,I Certain infectious and parasitic diseases (A...,1,"{'0.001': 0.99329, '0.01': 0.99733, '0.1': 0.9...",1101,891,253346,0,0,AB1_ANOGENITAL_HERPES_SIMPLEX,Anogenital herpesviral [herpes simplex] infection
2,[/mnt/nfs/pheweb/r6/generated-by-pheweb/pheno_...,I Certain infectious and parasitic diseases (A...,1,"{'0.001': 1.1145, '0.01': 1.0274, '0.1': 0.982...",1763,1450,258642,4,4,AB1_ARTHROPOD,Arthropod-borne viral fevers and viral haemorr...
3,[/mnt/nfs/pheweb/r6/generated-by-pheweb/pheno_...,I Certain infectious and parasitic diseases (A...,1,"{'0.001': 0.99661, '0.01': 1.0038, '0.1': 0.92...",106,87,255615,0,0,AB1_ASPERGILLOSIS,Aspergillosis
4,[/mnt/nfs/pheweb/r6/generated-by-pheweb/pheno_...,I Certain infectious and parasitic diseases (A...,1,"{'0.001': 0.99495, '0.01': 0.99733, '0.1': 0.9...",176,146,259010,0,0,AB1_ATYPICAL_CNS_VIRUS,Atypical virus infections of central nervous s...


In [16]:
import numpy as np
(
    pd.DataFrame(requests.get('https://r7.finngen.fi/api/phenos').json())
    .loc[lambda df: df.phenostring.isnull()]
)

,assoc_files,category,category_index,gc_lambda,num_cases,num_cases_prev,num_controls,num_gw_significant,num_gw_significant_prev,phenocode,phenostring


In [1]:
from pyspark.sql.window import Window


# from pyspark.sql.functions import row_number
windowSpec  = Window.partitionBy("study").orderBy("pval_text")


(
    spark.createDataFrame([
        {'study':'a', 'pval_text': None},
        {'study':'b', 'pval_text': 'pval 1'},
        {'study':'b', 'pval_text': 'pval 2'},
        {'study':'b', 'pval_text': None},
    ])
    .withColumn("row_number", f.row_number().over(windowSpec) - 1)
    .withColumn(
        "new_study",
        f.when(f.col('row_number') == 0, f.col('study')).otherwise(f.concat_ws('_', 'study', 'row_number'))
    )
    .drop('row_number')
    .show()
)

AttributeError: 'NoneType' object has no attribute '_jvm'

In [30]:
def make_new_study_id(df):
    ''' Creates a unique study ID from the origin STUDY ACCESSION for each
        new trait
    Params:
        df (pd.df): dataframe with 2 columns: Study accession, trait name
    Returns:
        list of strs
    '''
    cache = {}
    new_ids = []
    for accession, trait in df.values.tolist():

        # Add missing accession to cache
        if not accession in cache:
            cache[accession] = {'counter': 0}

        # Add missing trait to cache
        if not trait in cache[accession]:
            cache[accession]['counter'] += 1
            cache[accession][trait] = cache[accession]['counter']

        # Create new id. only add suffix if counter > 1
        if cache[accession][trait] == 1:
            new_id = accession
        elif cache[accession][trait] > 1:
            new_id = '{0}_{1}'.format(accession, cache[accession][trait])
        else:
            sys.exit('Error: counter < 1')
        new_ids.append(new_id)

    return new_ids

In [34]:
print(
    pd.DataFrame([
        {'study':'a', 'pval_text': None},
        {'study':'b', 'pval_text': 'pval 1'},
        {'study':'b', 'pval_text': 'pval 2'},
        {'study':'b', 'pval_text': None},
    ])
    .assign(new_study=lambda df: make_new_study_id(df))
    .to_markdown(index=False)
)


| study   | pval_text   | new_study   |
|:--------|:------------|:------------|
| a       |             | a           |
| b       | pval 1      | b           |
| b       | pval 2      | b_2         |
| b       |             | b_3         |


In [53]:
from random import randint, choice
from functools import reduce
import string

def random_char(y):
    """Generate random column names with a given lenght:"""
    return ''.join(choice(string.ascii_letters) for _ in range(y))

def get_dataframe():
    """Generate spark dataframe with some fixed and a random columns and random values"""
    randomcol = random_char(5)
    return spark.createDataFrame([
        {
            'a': randint(-1000, 1000), 
            'b':randint(-1000, 1000),
            randomcol : randint(-1000, 1000)
        } for _ in range(2)
    ])

# Generate a set of random dataframes:
dataframes = [get_dataframe() for _ in range(3)]

# Merging dataframes using reduce:
merged = reduce(lambda df1, df2: df1.unionByName(df2, allowMissingColumns=True), dataframes)
merged.show()

# +-----+----+----+-----+-----+
# |Nzbke|   a|   b|iFRbE|UtPKj|
# +-----+----+----+-----+-----+
# | -702| 994| 544| null| null|
# |    0| 700| 922| null| null|
# | null|-697| 434|  314| null|
# | null|-752| 165| -986| null|
# | null|-747|-227| null|  133|
# | null|-230| 517| null| -768|
# +-----+----+----+-----+-----+

In [55]:
merged.show()

+-----+----+----+-----+-----+
|Nzbke|   a|   b|iFRbE|UtPKj|
+-----+----+----+-----+-----+
| -702| 994| 544| null| null|
|    0| 700| 922| null| null|
| null|-697| 434|  314| null|
| null|-752| 165| -986| null|
| null|-747|-227| null|  133|
| null|-230| 517| null| -768|
+-----+----+----+-----+-----+



In [78]:
vl = (
    spark.read.json("/Users/dsuveges/repositories/evidence_datasource_parsers/validationLab_2022-05-20.json.gz")
    .select('diseaseFromSourceMappedId', 'targetFromSourceId')
    .withColumn('validationLab', f.lit(True))
    .distinct()
    .persist()
)

# vl.show()


ot = (
    spark.read.json('/Users/dsuveges/repositories/evidence_datasource_parsers/ot_crispr_2022-06-13.json.gz')
    .select('diseaseFromSourceMappedId', 'targetFromSourceId')
    .withColumn('otCrispr', f.lit(True))
    .persist()
)

gene_disease_pairs = (
    vl.select('diseaseFromSourceMappedId', 'targetFromSourceId')
    .distinct()
    .union(
        ot.select('diseaseFromSourceMappedId', 'targetFromSourceId').distinct()
    )
    .persist()
)

(
    vl.join(ot, on=['targetFromSourceId'], how='inner')
    .select('validationLab','otCrispr','targetFromSourceId')
    .distinct()
    .show(100)
)

+-------------+--------+------------------+
|validationLab|otCrispr|targetFromSourceId|
+-------------+--------+------------------+
|         true|    true|              PPIA|
|         true|    true|              HIRA|
|         true|    true|             KMT2D|
|         true|    true|             KDM2A|
|         true|    true|             CCNB1|
|         true|    true|            TADA2B|
|         true|    true|              RHOA|
|         true|    true|             CCDC6|
|         true|    true|             MYBL2|
|         true|    true|              WDR7|
|         true|    true|              USP7|
|         true|    true|              FADD|
|         true|    true|              PAK2|
|         true|    true|             TRAF2|
|         true|    true|            PRPF4B|
|         true|    true|             PDCD2|
|         true|    true|             USP10|
|         true|    true|             CCND1|
|         true|    true|           HSP90B1|
|         true|    true|        

In [84]:
(
    spark.read.parquet('/Users/dsuveges/project_data/molecule/')
    .select('id', 'name', 'tradeNames', 
        f.col('linkedTargets.rows').alias('linkedTargets'),
        f.explode(f.col('linkedDiseases.rows')).alias('diseaseFromSourceMappedId')
    )
    .select('*', f.explode(f.col('linkedTargets')).alias('targetFromSourceId'))
    .join(gene_disease_pairs, on=['diseaseFromSourceMappedId', 'targetFromSourceId'])
    .show(1, False, True)
)

(0 rows)



In [69]:
spark.read.parquet('/Users/dsuveges/project_data/molecule/').printSchema()

root
 |-- id: string (nullable = true)
 |-- canonicalSmiles: string (nullable = true)
 |-- inchiKey: string (nullable = true)
 |-- drugType: string (nullable = true)
 |-- blackBoxWarning: boolean (nullable = true)
 |-- name: string (nullable = true)
 |-- yearOfFirstApproval: long (nullable = true)
 |-- maximumClinicalTrialPhase: long (nullable = true)
 |-- parentId: string (nullable = true)
 |-- hasBeenWithdrawn: boolean (nullable = true)
 |-- isApproved: boolean (nullable = true)
 |-- withdrawnNotice: struct (nullable = true)
 |    |-- countries: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- classes: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- year: long (nullable = true)
 |-- tradeNames: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- synonyms: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- crossReferences: map (nullable = true)
 |    |-- key:

In [76]:
(
    vl.select('diseaseFromSourceMappedId', 'targetFromSourceId')
    .distinct()
    .union(
        ot.select('diseaseFromSourceMappedId', 'targetFromSourceId').distinct()
    )
).show()


+-------------------------+------------------+
|diseaseFromSourceMappedId|targetFromSourceId|
+-------------------------+------------------+
|              EFO_1001951|             GATA3|
|              EFO_1001951|            STAMBP|
|              EFO_1001951|              RHOA|
|              EFO_1001951|             FOXA1|
|              EFO_1001951|             SPDEF|
|              EFO_1001951|            PRPF4B|
|              EFO_1001951|             FDFT1|
|              EFO_1001951|             TRAF7|
|              EFO_1001951|             BRCA2|
|              EFO_1001951|           ARHGEF7|
|              EFO_1001951|              E2F3|
|              EFO_1001951|             KMT2D|
|              EFO_1001951|             FOSL1|
|              EFO_1001951|              USP7|
|              EFO_1001951|              FADD|
|              EFO_1001951|            TADA2B|
|              EFO_1001951|             CCDC6|
|              EFO_1001951|             UBE2C|
|            

In [85]:
df = pd.read_clipboard()
df.reshead()

,,Δ in number of evidence strings,Δ in number of invalid evidence strings,Δ in number of evidence strings dropped due to duplication,Δ in number of evidence strings dropped due to unresolved target,Δ in number of evidence strings dropped due to unresolved disease
,Nr of evidence strings in 22.04.3,,,,,
europepmc,9746463,-170392,0,0,0,0
crispr,1846,0,0,0,0,0
uniprot_variants,39419,-11035,-1414,-327,0,-1108
genomics_england,35622,-4824,-1449,10,-331,-1165
gene_burden,18715,-166,29,-18,-62,101


In [89]:
df2 = df.reset_index()

(
    df2
    .assign(
        
    )



,,Nr of evidence strings in 22.04.3,Δ in number of evidence strings,Δ in number of invalid evidence strings,Δ in number of evidence strings dropped due to duplication,Δ in number of evidence strings dropped due to unresolved target,Δ in number of evidence strings dropped due to unresolved disease
0,europepmc,9746463,-170392,0,0,0,0
1,crispr,1846,0,0,0,0,0
2,uniprot_variants,39419,-11035,-1414,-327,0,-1108
3,genomics_england,35622,-4824,-1449,10,-331,-1165
4,gene_burden,18715,-166,29,-18,-62,101
5,chembl,594375,-2218,-578,-425,0,-173
6,slapenrich,72369,53,-53,0,-53,0
7,expression_atlas,231165,0,0,0,0,0
8,intogen,3299,0,0,0,0,0
9,reactome,10345,0,0,0,0,0


- 22.04: 18.5, 22.06: 23.
- Combined set: 23.3k
- Not in 22.06: 320
- Not in 22.04: 4874
- New MONDO terms: 4720

In [91]:
d4 = spark.read.parquet('/Users/dsuveges/project_data/diseases_22.04/').persist()
d6 = spark.read.parquet('/Users/dsuveges/project_data/diseases_22.06/').persist()

print(d4.count())
print(d6.count())

18520
23074


In [96]:
joined = (
    d4
    .withColumnRenamed('id', 'id_04')
    .select('id_04', 'name')
    .join(d6.select('id', 'name'), on='name', how='outer')
    .persist()
)

In [102]:
(
    joined
    .filter(f.col('id_04').isNull())
    .filter(f.col('id').contains('MONDO'))
    .count()
)

4720

In [132]:
@f.udf(
    t.StructType([
        t.StructField('value', t.IntegerType()),
        t.StructField('array', t.StructType([
            t.StructField('a', t.IntegerType()),
            t.StructField('b', t.IntegerType()),
            t.StructField('c', t.IntegerType()),
        ]))
    ])
)
def get_column():
    return {'value': 4, 'array': [{'a': 93}, {'b': 58}, {'c': 36}] + {'d': choice([True, False])}}
#     return {
#         'value': randint(0,199),
#         'array': [{x: randint(0,100)} for x in ['a', 'b', 'c']]
#     }

df = (
    spark.createDataFrame([
        {'c1': 1}, {'c1': 2}, {'c1': 3}, {'c1': 4}
    ])
    .withColumn('cp', f.lit('cica'))
    .withColumn('complex_column', get_column())
    .persist()
)

df.show(truncate=False)

+---+----+-----------------------+
|c1 |cp  |complex_column         |
+---+----+-----------------------+
|1  |cica|{4, {null, null, null}}|
|2  |cica|{4, {null, null, null}}|
|3  |cica|{4, {null, null, null}}|
|4  |cica|{4, {null, null, null}}|
+---+----+-----------------------+



In [143]:
import pandas as pd

def get_column():
    return {'value': 4, 'array': [{'a': 93}, {'b': 58}, {'c': 36}]}
#     return 'pocok'

df = pd.DataFrame([{'column': get_column()} for _ in range(4)])
spark.createDataFrame(df).printSchema()

root
 |-- column: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)



In [144]:
from random import choice, randint
import pandas as pd
import json



In [247]:
from typing import Dict, List

# Preparing mock data:
def generate_entry() -> Dict[str, Dict[int, List[int]]]:
    data = {'col1': {'value_column': randint(1,1000)}}

    # Some times the array is not even there:
    if choice([True, True, False]):
        data['col1']['array_column'] = [generate_object() for _ in range(12)]

    return data

def generate_object() -> Dict[str, List[int]]:
    d = {x: randint(1,1000) for x in ['a', 'b', 'c']}
    d.update({'d': choice([True, False])})
    return d

# Saving data into a JSON (I couldn't resolve the issue otherwise):
with open('test_data2.json', 'wt') as test_file:
    for _ in range(12):
        json.dump(generate_entry(), test_file)
        test_file.write('\n')

# Reading data back as spark dataframe:
df = (
    spark.read.json('test_data2.json')
    # Adding lenght of the array:
    .withColumn(
        'array_lenght', 
        f.when(f.col('col1.array_column').isNotNull(), f.size(f.col('col1.array_column')))
        .otherwise(f.lit(0))
    )
    .withColumn('id', f.monotonically_increasing_id())
)

df.printSchema()
# root
#  |-- column: struct (nullable = true)
#  |    |-- array: array (nullable = true)
#  |    |    |-- element: struct (containsNull = true)
#  |    |    |    |-- a: long (nullable = true)
#  |    |    |    |-- b: long (nullable = true)
#  |    |    |    |-- c: long (nullable = true)
#  |    |    |    |-- d: boolean (nullable = true)
#  |    |-- value: long (nullable = true)


# Adding lenght of the array:
(
    df.show()
)

root
 |-- col1: struct (nullable = true)
 |    |-- array_column: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- a: long (nullable = true)
 |    |    |    |-- b: long (nullable = true)
 |    |    |    |-- c: long (nullable = true)
 |    |    |    |-- d: boolean (nullable = true)
 |    |-- value_column: long (nullable = true)
 |-- array_lenght: integer (nullable = false)
 |-- id: long (nullable = false)

+--------------------+------------+---+
|                col1|array_lenght| id|
+--------------------+------------+---+
|{[{664, 100, 900,...|          12|  0|
|{[{243, 417, 274,...|          12|  1|
|         {null, 614}|           0|  2|
|{[{631, 478, 423,...|          12|  3|
|{[{474, 716, 43, ...|          12|  4|
|{[{205, 459, 373,...|          12|  5|
|{[{345, 612, 785,...|          12|  6|
|{[{244, 792, 969,...|          12|  7|
|{[{281, 729, 56, ...|          12|  8|
|{[{301, 602, 677,...|          12|  9|
|{[{123, 986, 834,...|  

In [251]:
print(df.count())
xf = (
    df

    # Filtering arrays:
    .select(
        'id', 'array_lenght', f.col('col1.value_column').alias('value_column'),
        f.explode_outer('col1.array_column').alias('exploded')
    )
    .filter(
        (f.col('exploded.d') == True)
        | f.col('exploded').isNull()
    )
    
    # Aggregating stuff:
    .groupBy('id', 'array_lenght')
    .agg(
        f.struct(
            f.collect_set(f.col('exploded')).alias('array_column'),
            f.first(f.col('value_column')).alias('value_column')
        ).alias('col1')
    )
    .withColumn('new_count', f.size(f.col('col1.array_column')))
    .orderBy('id')
)
print(xf.show())

12
+---+------------+--------------------+---------+
| id|array_lenght|                col1|new_count|
+---+------------+--------------------+---------+
|  0|          12|{[{417, 937, 422,...|        6|
|  1|          12|{[{700, 722, 785,...|        6|
|  2|           0|           {[], 614}|        0|
|  3|          12|{[{237, 739, 174,...|        4|
|  4|          12|{[{93, 564, 532, ...|        7|
|  5|          12|{[{233, 322, 520,...|        9|
|  6|          12|{[{481, 647, 434,...|        4|
|  7|          12|{[{475, 8, 987, t...|        4|
|  8|          12|{[{394, 122, 353,...|        6|
|  9|          12|{[{197, 100, 77, ...|        6|
| 10|          12|{[{885, 764, 701,...|        3|
| 11|          12|{[{855, 297, 11, ...|        4|
+---+------------+--------------------+---------+

None


In [249]:
# Processing array columns:
array_cols = (
    df.select('id', f.explode(f.col('col1.array_column')).alias('exploded'))
    .filter(f.col('exploded.d') == True)
    .groupby('id')
    .agg(f.collect_list(f.col('exploded')).alias('array_column'))
)

# Joining with original data:
(
    df
    .join(array_cols, on='id', how='left')
    .withColumn('col1', f.struct(f.col('col1.value_column'), f.col('array_column')))
    .drop('array_column')
    .withColumn(
        'new_length', 
        f.when(f.col('col1.array_column').isNotNull(), f.size(f.col('col1.array_column')))
        .otherwise(f.lit(0))
    )
    .show()
)

+---+--------------------+------------+----------+
| id|                col1|array_lenght|new_length|
+---+--------------------+------------+----------+
|  0|{372, [{664, 100,...|          12|         6|
|  1|{828, [{228, 387,...|          12|         6|
|  2|         {614, null}|           0|         0|
|  3|{877, [{881, 714,...|          12|         4|
|  4|{595, [{474, 716,...|          12|         7|
|  5|{554, [{233, 322,...|          12|         9|
|  6|{877, [{481, 647,...|          12|         4|
|  7|{881, [{475, 8, 9...|          12|         4|
|  8|{905, [{281, 729,...|          12|         6|
|  9|{142, [{301, 602,...|          12|         6|
| 10|{277, [{854, 226,...|          12|         3|
| 11|{576, [{350, 635,...|          12|         4|
+---+--------------------+------------+----------+



In [265]:
(
    df
    .select("*", f.expr("filter(col1.array_column, array -> array.d == True)").alias("filtered_arrays"))
    .show()
)

+--------------------+------------+---+--------------------+-----+
|                col1|array_lenght| id|     filtered_arrays|count|
+--------------------+------------+---+--------------------+-----+
|{[{664, 100, 900,...|          12|  0|[{664, 100, 900, ...|    6|
|{[{243, 417, 274,...|          12|  1|[{228, 387, 656, ...|    6|
|         {null, 614}|           0|  2|                null|   -1|
|{[{631, 478, 423,...|          12|  3|[{881, 714, 234, ...|    4|
|{[{474, 716, 43, ...|          12|  4|[{474, 716, 43, t...|    7|
|{[{205, 459, 373,...|          12|  5|[{233, 322, 520, ...|    9|
|{[{345, 612, 785,...|          12|  6|[{481, 647, 434, ...|    4|
|{[{244, 792, 969,...|          12|  7|[{475, 8, 987, tr...|    4|
|{[{281, 729, 56, ...|          12|  8|[{281, 729, 56, t...|    6|
|{[{301, 602, 677,...|          12|  9|[{301, 602, 677, ...|    6|
|{[{123, 986, 834,...|          12| 10|[{854, 226, 406, ...|    3|
|{[{763, 821, 428,...|          12| 11|[{350, 635, 274, ...|  

In [266]:
import pandas as pd


def explode(df, columns):
    ''' Explodes multiple columns
    '''
    idx = np.repeat(df.index, df[columns[0]].str.len())
    a = df.T.reindex(columns).values
    concat = np.concatenate([np.concatenate(a[i]) for i in range(a.shape[0])])
    p = pd.DataFrame(concat.reshape(a.shape[0], -1).T, idx, columns)
    return pd.concat([df.drop(columns, axis=1), p], axis=1).reset_index(drop=True)




In [276]:
d = (
    pd.DataFrame({
        'a': [1,2,3,4,5],
        'b': ['c,h','d,i', 'e,j', 'f,k', 'g'],
        'c': ['l,m','o,z', 'p,q', 's,t', 'v']
    })
    .assign(
        b = lambda df: df.b.str.split(','),
        c = lambda df: df.c.str.split(',')
    )
)

explode(d, ['b', 'c'])

,a,b,c
0,1,c,l
1,1,h,m
2,2,d,o
3,2,i,z
4,3,e,p
5,3,j,q
6,4,f,s
7,4,k,t
8,5,g,v


In [280]:
(
    spark.createDataFrame([{"COLUMN NAME":1, "COLUMN NAME 2": 4}])
    .transform(lambda df: df.select(*[f.col(c).alias(c.lower().replace(' ', '_')) for c in df.columns]))
    .show()
)

+-----------+-------------+
|column_name|column_name_2|
+-----------+-------------+
|          1|            4|
+-----------+-------------+



In [281]:
import quinn
(
    spark.createDataFrame([{"COLUMN NAME":1, "COLUMN NAME 2": 4}])
    .transform(quinn.snake_case_col_names)
    .show()
)

+-----------+-------------+
|column_name|column_name_2|
+-----------+-------------+
|          1|            4|
+-----------+-------------+



In [287]:
%%bash

ls -lah

total 1746032
drwxr-xr-x  17 dsuveges  384566875   544B 26 Jun 22:52 .
drwxrwxr-x  85 dsuveges  384566875   2.7K 15 Jun 14:30 ..
drwxr-xr-x   5 dsuveges  384566875   160B 24 May 23:22 .ipynb_checkpoints
-rw-r--r--   1 dsuveges  384566875   252K 20 Jun 14:58 New_layout.ipynb
-rw-r--r--   1 dsuveges  384566875   190K 26 Jun 22:50 annotate_gwascat_varaintids.ipynb
-rw-r--r--   1 dsuveges  384566875   164M 26 Jun 22:43 gwas-catalog-associations.tsv
-rw-r--r--   1 dsuveges  384566875    48M 26 Jun 22:51 gwas-catalog-associations.tsv.1
-rw-r--r--   1 dsuveges  384566875    11M 27 May 09:49 gwas_ancestries_new_2022-05-27.tsv
-rw-r--r--   1 dsuveges  384566875   164M 26 Jun 22:53 gwas_assoc_22_01_26.tsv
-rw-r--r--   1 dsuveges  384566875   191M 26 Jun 22:52 gwas_assoc_22_06_16.tsv
-rw-r--r--   1 dsuveges  384566875   229M 27 May 09:49 gwas_associations_2022-05-27.tsv
-rw-r--r--   1 dsuveges  384566875    19M 27 May 09:49 gwas_studies_new_2022-05-27.tsv
-rw-r--r--   1 dsuveges  384566875   741K

In [305]:
files = {
    'gwas_assoc_22_01_26.tsv': 'Jan_2022',
    'gwas_assoc_22_06_16.tsv': 'Jun_2022',
    'gwas_assoc_21_06_08.tsv': 'Jun_2021'
}


def read_data(file, date):
    return (
        spark.read.csv(file, sep='\t', header=True)
        .filter(f.col('SNPS').startswith('rs'))
        .withColumn(f'{date}_mapped', f.when(f.col('CHR_ID').isNotNull(), f.lit(True)).otherwise(f.lit(False)))
        .select(
            f.col('SNPS').alias('rsIds'),
            f.col('PUBMEDID').alias('pmid'),
            f'{date}_mapped'
        )
        .distinct()
        .persist()
    )
    
dataframes = [read_data(file, date) for file, date in files.items()]

In [307]:

from functools import reduce
df = reduce(lambda df1, df2: df1.join(df2, on=['pmid', 'rsIds'], how='inner').persist(), dataframes)

df.show()

+--------+--------------------+---------------+---------------+---------------+
|    pmid|               rsIds|Jan_2022_mapped|Jun_2022_mapped|Jun_2021_mapped|
+--------+--------------------+---------------+---------------+---------------+
|26634245|         rs142412198|           true|           true|           true|
|28604732|           rs2713206|           true|           true|           true|
|27082954|          rs11012350|           true|           true|           true|
|28928442|           rs4360170|           true|           true|           true|
|29292387|           rs4955417|           true|           true|           true|
|28928442|         rs139260327|           true|           true|           true|
|28429243|          rs57712673|           true|           true|           true|
|28448500|          rs10101061|           true|           true|           true|
|27903959|rs1547399; rs2213...|           true|           true|           true|
|29472613|          rs17753687|         

In [317]:
@f.udf(t.BooleanType())
def is_concordant(b1: bool, b2: bool, b3: bool) -> bool:
    if b1 == b2 == b3:
        return True
    else:
        return False
annotated_df = (
    df
    .withColumn('is_concordant', is_concordant(f.col('Jan_2022_mapped'),
                                               f.col('Jun_2022_mapped'),
                                                 f.col('Jun_2021_mapped')))
    .persist()
)

In [336]:
(
    annotated_df
    .filter((f.col('is_concordant') == True) & (f.col('Jan_2022_mapped') == False))
    .show()
)
print(
    annotated_df
    .filter((f.col('is_concordant') == True) & (f.col('Jan_2022_mapped') == False))
    .count()
)

+--------+-------------+---------------+---------------+---------------+-------------+
|    pmid|        rsIds|Jan_2022_mapped|Jun_2022_mapped|Jun_2021_mapped|is_concordant|
+--------+-------------+---------------+---------------+---------------+-------------+
|29874175|  rs1042391-T|          false|          false|          false|         true|
|29875488|  rs201274146|          false|          false|          false|         true|
|25188341|    rs9406351|          false|          false|          false|         true|
|29077507|   rs35077018|          false|          false|          false|         true|
|25576479| rs10518889-A|          false|          false|          false|         true|
|31043758|   rs2428362†|          false|          false|          false|         true|
|25576479|rs115231908-C|          false|          false|          false|         true|
|25576479| rs10445216-C|          false|          false|          false|         true|
|31740837|   rs13004237|          false|   

In [321]:
(
    annotated_df
    .filter(f.col('pmid') == '29874175')
    .show()
)

+--------+------------+---------------+---------------+---------------+-------------+
|    pmid|       rsIds|Jan_2022_mapped|Jun_2022_mapped|Jun_2021_mapped|is_concordant|
+--------+------------+---------------+---------------+---------------+-------------+
|29874175| rs1042391-T|          false|          false|          false|         true|
|29874175| rs7188697-G|          false|          false|          false|         true|
|29874175| rs4934956-T|          false|          false|          false|         true|
|29874175| rs2074518-A|          false|          false|          false|         true|
|29874175| rs1361754-G|          false|          false|          false|         true|
|29874175| rs7439869-T|          false|          false|          false|         true|
|29874175| rs3189030-A|          false|          false|          false|         true|
|29874175|rs55910611-A|          false|          false|          false|         true|
|29874175|rs17608766-C|          false|          false

In [326]:
(
    spark.read.csv(file, sep='\t', header=True)
    .filter(f.col('PUBMEDID') == '29874175')
    .select('PUBMEDID', 'SNPS', 'STRONGEST SNP-RISK ALLELE', 'CHR_ID', 'CHR_POS')
    .show()
)

+--------+------------+-------------------------+------+-------+
|PUBMEDID|        SNPS|STRONGEST SNP-RISK ALLELE|CHR_ID|CHR_POS|
+--------+------------+-------------------------+------+-------+
|29874175| rs7188697-G|                rs7188697|  null|   null|
|29874175| rs2074518-A|                rs2074518|  null|   null|
|29874175| rs9912468-G|                rs9912468|  null|   null|
|29874175|rs17779747-T|               rs17779747|  null|   null|
|29874175| rs6762208-A|                rs6762208|  null|   null|
|29874175|  rs737154-C|                 rs737154|  null|   null|
|29874175| rs9470361-A|                rs9470361|  null|   null|
|29874175| rs2926743-A|                rs2926743|  null|   null|
|29874175| rs1361754-G|                rs1361754|  null|   null|
|29874175|rs55910611-A|               rs55910611|  null|   null|
|29874175| rs1801725-T|                rs1801725|  null|   null|
|29874175| rs4934956-T|                rs4934956|  null|   null|
|29874175| rs3189030-A|  

In [334]:
(
    spark.read.csv(file, sep='\t', header=True)
    .filter(f.col('SNPS').contains('-') & f.col('SNPS').startswith('rs'))
    .select('PUBMEDID', 'SNPS', 'STRONGEST SNP-RISK ALLELE', 'CHR_ID', 'CHR_POS')
    .orderBy('PUBMEDID')
    .show(1000)
)

+--------+-------------+-------------------------+------+-------+
|PUBMEDID|         SNPS|STRONGEST SNP-RISK ALLELE|CHR_ID|CHR_POS|
+--------+-------------+-------------------------+------+-------+
|25576479|rs114539181-G|              rs114539181|  null|   null|
|25576479|rs115231908-C|              rs115231908|  null|   null|
|25576479| rs10518889-A|               rs10518889|  null|   null|
|25576479|rs114747981-C|              rs114747981|  null|   null|
|25576479|    rs12938-C|                  rs12938|  null|   null|
|25576479|rs116715409-G|              rs116715409|  null|   null|
|25576479|  rs3917768-C|                rs3917768|  null|   null|
|25576479|rs116121798-A|              rs116121798|  null|   null|
|25576479|    rs12938-C|                  rs12938|  null|   null|
|25576479| rs11632650-C|               rs11632650|  null|   null|
|25576479|   rs228966-A|                 rs228966|  null|   null|
|25576479|    rs12938-C|                  rs12938|  null|   null|
|25576479|

In [362]:

(
    spark.read.csv(file, sep='\t', header=True)
    .filter(f.col('SNPS').rlike(r'rs\d+\D+'))
    .filter(~ f.col('SNPS').rlike(r'[x;]'))
    .filter(f.col('CHR_ID').isNull())
    .withColumn('SNPS', f.udf(lambda x: f'"{x}"', t.StringType())(f.col('SNPS')))
    .select('PUBMEDID', 'SNPS', 'STRONGEST SNP-RISK ALLELE', 'CHR_ID', 'CHR_POS')
    .orderBy('PUBMEDID')
    .distinct()
    .show(500, truncate=False)
)



+--------+------------------------+---------------------------+------+-------+
|PUBMEDID|SNPS                    |STRONGEST SNP-RISK ALLELE  |CHR_ID|CHR_POS|
+--------+------------------------+---------------------------+------+-------+
|32193507|"rs2630445a"            |rs2630445a-T               |null  |null   |
|29874175|"rs1535-G"              |rs1535                     |null  |null   |
|31251335|"rs41274896 "           |rs41274896 -G              |null  |null   |
|29874175|"rs8049607-T"           |rs8049607                  |null  |null   |
|29874175|"rs737154-C"            |rs737154                   |null  |null   |
|25576479|"rs621437-C"            |rs621437                   |null  |null   |
|30573740|"rs11639303_G_A"        |rs11639303_G_A-A           |null  |null   |
|29874175|"rs6762208-A"           |rs6762208                  |null  |null   |
|31251335|"rs10179193 "           |rs10179193 -C              |null  |null   |
|31251335|"rs857958 "             |rs857958 -T      

In [395]:
(
    spark.read.csv(file, sep='\t', header=True)
#     .filter(f.col('MERGED') == 1)
    .filter(~f.col('SNP_ID_CURRENT').rlike(r'^\d+$'))
#     .withColumn('cicaful', f.format_string('rs%s', f.col('SNP_ID_CURRENT')))
#     .filter(f.col('SNP_ID_CURRENT').isNotNull())
#     .filter(f.concat(f.lit('rs') + f.col('SNP_ID_CURRENT')) != f.col('SNPS'))
    .select(f.format_string('"%s"', f.col('SNP_ID_CURRENT')), 'SNPS')
#     .show(100)
    .count()
)

99

In [397]:
bases = ('ACTG', 'TGAC')
tr = {ord(a):ord(b) for a, b in zip(*bases)}

In [401]:
spark.createDataFrame([('TTGC',)], ['a']).select('a', f.translate('a', "ACTG", "TGAC").alias('r')).show()

+----+----+
|   a|   r|
+----+----+
|TTGC|AACG|
+----+----+

